In [3]:
import torch
print("Kaggle test run successful")


Kaggle test run successful


In [4]:
!pip install timm==0.4.12 --no-cache-dir


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.0/377.0 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 162.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 187.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 167.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 183.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 179.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 162.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 148.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 106.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 134.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 122.5 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: nvid

In [6]:
!pip install continuum>=1.0.27

In [2]:
from PIL import Image
import os
from torch.utils.data import Dataset

class CDDBBinaryDataset(Dataset):
    def __init__(self, root, task, split, transform):
        self.root = root
        self.task = task
        self.split = split
        self.transform = transform
        self.task_id = task_id

        self.data = []
        self.targets = []
        self.task_ids = []

        # Path setup
        task_folder = os.path.join(root, task)
        nested_folder = os.path.join(task_folder, task)
        if os.path.exists(nested_folder):
            task_folder = nested_folder

        task_folder = os.path.join(task_folder, split)
        if not os.path.exists(task_folder):
            print(f"[Warning] Task folder not found: {task_folder}")
            return

        # Load images
        if task.lower() == "cyclegan":
            for subfolder in os.listdir(task_folder):
                subfolder_path = os.path.join(task_folder, subfolder)
                if not os.path.isdir(subfolder_path):
                    continue
                for sub, label in [("0_real", 0), ("1_fake", 1)]:
                    label_folder = os.path.join(subfolder_path, sub)
                    if not os.path.exists(label_folder):
                        continue
                    for img_name in os.listdir(label_folder):
                        if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                            self.data.append(os.path.join(label_folder, img_name))
                            self.targets.append(label)
                            self.task_ids.append(task_id)
        else:
            for sub, label in [("0_real", 0), ("1_fake", 1)]:
                label_folder = os.path.join(task_folder, sub)
                if not os.path.exists(label_folder):
                    continue
                for img_name in os.listdir(label_folder):
                    if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                        self.data.append(os.path.join(label_folder, img_name))
                        self.targets.append(label)
                        self.task_ids.append(task_id)

        # For compatibility with Memory class we'll expose these attributes
        # as arrays/lists. Memory may replace these later.
        self._x = list(self.data)
        self._y = list(self.targets)
        self._t = list(self.task_ids)

        print(f"[INFO] Loaded {len(self._x)} images for task '{task}' ({split})")

    def __len__(self):
        return len(self._x)

    def __getitem__(self, idx):
        """
        Returns (image_tensor, label, task_id)
        - This method uses self._x/_y/_t if present, otherwise falls back to internal lists.
        - If self._x contains image *paths*, it opens them on the fly (lazy load).
        - If self._x contains preloaded tensors/arrays, returns them directly.
        """
        # Prefer _x/_y/_t (these may be set by Memory.get_dataset)
        if hasattr(self, '_x') and self._x is not None:
            x = self._x[idx]
            y = self._y[idx]
            t = self._t[idx]
        else:
            x = self.data[idx]
            y = self.targets[idx]
            t = self.task_ids[idx]

        # If x is a path-like string, load image
        if isinstance(x, str) or (hasattr(x, 'endswith') and callable(x.endswith)):
            img = Image.open(x).convert("RGB")
            if self.transform:
                img = self.transform(img)
        else:
            # assume tensor/ndarray already
            img = x
            if self.transform and not isinstance(img, torch.Tensor):
                # usually transform expects PIL, so skip transform here if already tensor
                pass

        return img, y




In [3]:
from torchvision import transforms

train_transform = transforms.Compose([
   transforms.RandomResizedCrop(224, scale=(0.7, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.4, 0.4, 0.4, 0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

In [4]:
from torch.utils.data import DataLoader
tasks = ["gaugan","biggan","cyclegan","imle","deepfake","crn"]  # EASY sequence
root = "/kaggle/input/cddb-easy"

task_loaders = []
val_loaders = []
for task_id, task in enumerate(tasks):
    
    train_dataset = CDDBBinaryDataset(root=root, task=task, split="train", transform=train_transform)
    val_dataset = CDDBBinaryDataset(root=root, task=task, split="val", transform=val_transform)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

    task_loaders.append(train_loader)
    val_loaders.append(val_loader)


[INFO] Loaded 6000 images for task 'gaugan' (train)
[INFO] Loaded 2000 images for task 'gaugan' (val)
[INFO] Loaded 2400 images for task 'biggan' (train)
[INFO] Loaded 800 images for task 'biggan' (val)
[INFO] Loaded 1572 images for task 'cyclegan' (train)
[INFO] Loaded 524 images for task 'cyclegan' (val)
[INFO] Loaded 7656 images for task 'imle' (train)
[INFO] Loaded 2552 images for task 'imle' (val)
[INFO] Loaded 3248 images for task 'deepfake' (train)
[INFO] Loaded 1082 images for task 'deepfake' (val)
[INFO] Loaded 7658 images for task 'crn' (train)
[INFO] Loaded 2552 images for task 'crn' (val)


In [10]:
import os

# Create your custom folder
os.makedirs("/kaggle/working/continual", exist_ok=True)
print("Created folder:", os.listdir("/kaggle/working"))


Created folder: ['.virtual_documents', '=1.0.27', 'continual']


In [11]:
# Write Python code to a new file
code = """

import io
import os
import time
from collections import defaultdict, deque
import datetime
import warnings

import torch
from torch import nn
import torch.distributed as dist


class SmoothedValue(object):
  

    def __init__(self, window_size=20, fmt=None):
        if fmt is None:
            fmt = "{median:.4f} ({global_avg:.4f})"
        self.deque = deque(maxlen=window_size)
        self.total = 0.0
        self.count = 0
        self.fmt = fmt

    def update(self, value, n=1):
        self.deque.append(value)
        self.count += n
        self.total += value * n

    def synchronize_between_processes(self):
       
        if not is_dist_avail_and_initialized():
            return
        t = torch.tensor([self.count, self.total], dtype=torch.float64, device='cuda')
        dist.barrier()
        dist.all_reduce(t)
        t = t.tolist()
        self.count = int(t[0])
        self.total = t[1]

    @property
    def median(self):
        d = torch.tensor(list(self.deque))
        return d.median().item()

    @property
    def avg(self):
        d = torch.tensor(list(self.deque), dtype=torch.float32)
        return d.mean().item()

    @property
    def global_avg(self):
        return self.total / self.count

    @property
    def max(self):
        return max(self.deque)

    @property
    def value(self):
        return self.deque[-1]

    def __str__(self):
        return self.fmt.format(
            median=self.median,
            avg=self.avg,
            global_avg=self.global_avg,
            max=self.max,
            value=self.value)


class MetricLogger(object):
    def __init__(self, delimiter="\t"):
        self.meters = defaultdict(SmoothedValue)
        self.delimiter = delimiter

    def update(self, **kwargs):
        for k, v in kwargs.items():
            if v is None:
                continue
            if isinstance(v, torch.Tensor):
                v = v.item()
            assert isinstance(v, (float, int))
            self.meters[k].update(v)

    def update_dict(self, d):
        for k, v in d.items():
            if isinstance(v, torch.Tensor):
                v = v.item()
            assert isinstance(v, (float, int))
            self.meters[k].update(v)

    def __getattr__(self, attr):
        if attr in self.meters:
            return self.meters[attr]
        if attr in self.__dict__:
            return self.__dict__[attr]
        raise AttributeError("'{}' object has no attribute '{}'".format(
            type(self).__name__, attr))

    def __str__(self):
        loss_str = []
        for name, meter in self.meters.items():
            loss_str.append(
                "{}: {}".format(name, str(meter))
            )
        return self.delimiter.join(loss_str)

    def synchronize_between_processes(self):
        for meter in self.meters.values():
            meter.synchronize_between_processes()

    def add_meter(self, name, meter):
        self.meters[name] = meter

    def log_every(self, iterable, print_freq, header=None):
        i = 0
        if not header:
            header = ''
        start_time = time.time()
        end = time.time()
        iter_time = SmoothedValue(fmt='{avg:.4f}')
        data_time = SmoothedValue(fmt='{avg:.4f}')
        space_fmt = ':' + str(len(str(len(iterable)))) + 'd'
        log_msg = [
            header,
            '[{0' + space_fmt + '}/{1}]',
            'eta: {eta}',
            '{meters}',
            'time: {time}',
            'data: {data}'
        ]
        if torch.cuda.is_available():
            log_msg.append('max mem: {memory:.0f}')
        log_msg = self.delimiter.join(log_msg)
        MB = 1024.0 * 1024.0
        for obj in iterable:
            data_time.update(time.time() - end)
            yield obj
            iter_time.update(time.time() - end)
            if i % print_freq == 0 or i == len(iterable) - 1:
                eta_seconds = iter_time.global_avg * (len(iterable) - i)
                eta_string = str(datetime.timedelta(seconds=int(eta_seconds)))
                if torch.cuda.is_available():
                    print(log_msg.format(
                        i, len(iterable), eta=eta_string,
                        meters=str(self),
                        time=str(iter_time), data=str(data_time),
                        memory=torch.cuda.max_memory_allocated() / MB))
                else:
                    print(log_msg.format(
                        i, len(iterable), eta=eta_string,
                        meters=str(self),
                        time=str(iter_time), data=str(data_time)))
            i += 1
            end = time.time()
        total_time = time.time() - start_time
        total_time_str = str(datetime.timedelta(seconds=int(total_time)))
        print('{} Total time: {} ({:.4f} s / it)'.format(
            header, total_time_str, total_time / len(iterable)))


def _load_checkpoint_for_ema(model_ema, checkpoint):
    
    mem_file = io.BytesIO()
    torch.save(checkpoint, mem_file)
    mem_file.seek(0)
    model_ema._load_checkpoint(mem_file)


def setup_for_distributed(is_master):
 
    import builtins as __builtin__
    builtin_print = __builtin__.print

    def print(*args, **kwargs):
        force = kwargs.pop('force', False)
        if is_master or force:
            builtin_print(*args, **kwargs)

    __builtin__.print = print


def is_dist_avail_and_initialized():
    if not dist.is_available():
        return False
    if not dist.is_initialized():
        return False
    return True


def get_world_size():
    if not is_dist_avail_and_initialized():
        return 1
    return dist.get_world_size()


def get_rank():
    if not is_dist_avail_and_initialized():
        return 0
    return dist.get_rank()


def is_main_process():
    return get_rank() == 0


def save_on_master(*args, **kwargs):
    if is_main_process():
        torch.save(*args, **kwargs)


def init_distributed_mode(args):
    if 'RANK' in os.environ and 'WORLD_SIZE' in os.environ:
        args.rank = int(os.environ["RANK"])
        args.world_size = int(os.environ['WORLD_SIZE'])
        args.gpu = int(os.environ['LOCAL_RANK'])
    #elif 'SLURM_PROCID' in os.environ:
    #    args.rank = int(os.environ['SLURM_PROCID'])
    #    args.gpu = args.rank % torch.cuda.device_count()
    else:
        print('Not using distributed mode')
        args.distributed = False
        return

    args.distributed = True

    torch.cuda.set_device(args.gpu)
    args.dist_backend = 'nccl'
    print('| distributed init (rank {}): {}'.format(
        args.rank, args.dist_url), flush=True)
    torch.distributed.init_process_group(backend=args.dist_backend, init_method=args.dist_url,
                                         world_size=args.world_size, rank=args.rank,
                                         timeout=datetime.timedelta(hours=2))
    torch.distributed.barrier()
    setup_for_distributed(args.rank == 0)


def load_first_task_model(model_without_ddp, loss_scaler, task_id, args):
    strict = False

    if args.resume.startswith('https'):
        checkpoint = torch.hub.load_state_dict_from_url(
            args.resume, map_location='cpu', check_hash=True)
    elif os.path.isdir(args.resume):
        path = os.path.join(args.resume, f"checkpoint_{task_id}.pth")
        checkpoint = torch.load(path, map_location='cpu')
    else:
        checkpoint = torch.load(args.resume, map_location='cpu')

    model_ckpt = checkpoint['model']

    if not strict:
        for i in range(1, 6):
            k = f"head.fcs.{i}.weight"
            if k in model_ckpt: del model_ckpt[k]
            k = f"head.fcs.{i}.bias"
            if k in model_ckpt: del model_ckpt[k]
    model_without_ddp.load_state_dict(model_ckpt, strict=strict)
    if not args.eval and 'optimizer' in checkpoint and 'lr_scheduler' in checkpoint and 'epoch' in checkpoint:
        #optimizer.load_state_dict(checkpoint['optimizer'])
        #lr_scheduler.load_state_dict(checkpoint['lr_scheduler'])
        #args.start_epoch = checkpoint['epoch'] + 1
        #if args.model_ema:
        #    utils._load_checkpoint_for_ema(model_ema, checkpoint['model_ema'])
        if 'scaler' in checkpoint:
            try:
                loss_scaler.load_state_dict(checkpoint['scaler'])
            except:
                warnings.warn("Could not reload loss scaler, probably because of amp/noamp mismatch")



def change_pos_embed_size(pos_embed, new_size=32, patch_size=16, old_size=224):
    nb_patches = (new_size // patch_size) ** 2
    new_pos_embed = torch.randn(1, nb_patches + 1, pos_embed.shape[2])
    new_pos_embed[0, 0] = pos_embed[0, 0]

    lo_idx = 1
    for i in range(nb_patches):
        hi_idx = lo_idx + old_size // nb_patches
        new_pos_embed[0, i] = pos_embed[0, lo_idx:hi_idx].mean(dim=0)
        lo_idx = hi_idx

    return torch.nn.Parameter(new_pos_embed)

def freeze_parameters(m, requires_grad=False):
    if m is None:
        return

    if isinstance(m, nn.Parameter):
        m.requires_grad = requires_grad
    else:
        for p in m.parameters():
            p.requires_grad = requires_grad
"""

with open("/kaggle/working/continual/utils.py", "w") as f:
    f.write(code)

print("File created!")


File created!


In [6]:
import copy
import math
from functools import lru_cache

import torch
import torch.nn as nn
import torch.nn.functional as F
from timm.models.layers import DropPath, to_2tuple, trunc_normal_

import continual.utils as cutils


class BatchEnsemble(nn.Module):
    def __init__(self, in_features, out_features, bias=True):
        super().__init__()

        self.linear = nn.Linear(in_features, out_features, bias=bias)

        self.out_features, self.in_features = out_features, in_features
        self.bias = bias

        self.r = nn.Parameter(torch.randn(self.out_features))
        self.s = nn.Parameter(torch.randn(self.in_features))

    def __deepcopy__(self, memo):
        cls = self.__class__
        result = cls.__new__(cls, self.in_features, self.out_features, self.bias)
        memo[id(self)] = result
        for k, v in self.__dict__.items():
            setattr(result, k, copy.deepcopy(v, memo))

        result.linear.weight = self.linear.weight
        return result

    def reset_parameters(self):
        device = self.linear.weight.device
        self.r = nn.Parameter(torch.randn(self.out_features).to(device))
        self.s = nn.Parameter(torch.randn(self.in_features).to(device))

        if self.bias:
            fan_in, _ = nn.init._calculate_fan_in_and_fan_out(self.linear.weight)
            bound = 1 / math.sqrt(fan_in) if fan_in > 0 else 0
            nn.init.uniform_(self.linear.bias, -bound, bound)

    def forward(self, x):
        w = torch.outer(self.r, self.s)
        w = w * self.linear.weight
        return F.linear(x, w, self.linear.bias)


class Mlp(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU, drop=0., fc=nn.Linear):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = fc(in_features, hidden_features)
        self.act = act_layer()
        self.fc2 = fc(hidden_features, out_features)
        self.drop = nn.Dropout(drop)
        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, BatchEnsemble):
            trunc_normal_(m.linear.weight, std=.02)
            if isinstance(m.linear, nn.Linear) and m.linear.bias is not None:
                nn.init.constant_(m.linear.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x


class GPSA(nn.Module):
    def __init__(self, dim, num_heads=8, qkv_bias=False, qk_scale=None, attn_drop=0., proj_drop=0.,
                 locality_strength=1., use_local_init=True, fc=None):
        super().__init__()
        self.num_heads = num_heads
        self.dim = dim
        head_dim = dim // num_heads
        self.scale = qk_scale or head_dim ** -0.5

        self.qk = nn.Linear(dim, dim * 2, bias=qkv_bias)
        self.v = nn.Linear(dim, dim, bias=qkv_bias)

        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.pos_proj = nn.Linear(3, num_heads)
        self.proj_drop = nn.Dropout(proj_drop)
        self.locality_strength = locality_strength
        self.gating_param = nn.Parameter(torch.ones(self.num_heads))
        self.apply(self._init_weights)
        if use_local_init:
            self.local_init(locality_strength=locality_strength)

    def reset_parameters(self):
        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    def forward(self, x):
        B, N, C = x.shape
        if not hasattr(self, 'rel_indices') or self.rel_indices.size(1)!=N:
            self.get_rel_indices(N)

        attn = self.get_attention(x)
        v = self.v(x).reshape(B, N, self.num_heads, C // self.num_heads).permute(0, 2, 1, 3)
        x = (attn @ v).transpose(1, 2).reshape(B, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x, attn, v

    def get_attention(self, x):
        B, N, C = x.shape
        qk = self.qk(x).reshape(B, N, 2, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        q, k = qk[0], qk[1]
        pos_score = self.rel_indices.expand(B, -1, -1,-1)
        pos_score = self.pos_proj(pos_score).permute(0,3,1,2)
        patch_score = (q @ k.transpose(-2, -1)) * self.scale
        patch_score = patch_score.softmax(dim=-1)
        pos_score = pos_score.softmax(dim=-1)

        gating = self.gating_param.view(1,-1,1,1)
        attn = (1.-torch.sigmoid(gating)) * patch_score + torch.sigmoid(gating) * pos_score
        attn /= attn.sum(dim=-1).unsqueeze(-1)
        attn = self.attn_drop(attn)
        return attn

    def get_attention_map(self, x, return_map = False):

        attn_map = self.get_attention(x).mean(0) # average over batch
        distances = self.rel_indices.squeeze()[:,:,-1]**.5
        dist = torch.einsum('nm,hnm->h', (distances, attn_map))
        dist /= distances.size(0)
        if return_map:
            return dist, attn_map
        else:
            return dist

    def local_init(self, locality_strength=1.):
        self.v.weight.data.copy_(torch.eye(self.dim))
        locality_distance = 1 #max(1,1/locality_strength**.5)

        kernel_size = int(self.num_heads**.5)
        center = (kernel_size-1)/2 if kernel_size%2==0 else kernel_size//2
        for h1 in range(kernel_size):
            for h2 in range(kernel_size):
                position = h1+kernel_size*h2
                self.pos_proj.weight.data[position,2] = -1
                self.pos_proj.weight.data[position,1] = 2*(h1-center)*locality_distance
                self.pos_proj.weight.data[position,0] = 2*(h2-center)*locality_distance
        self.pos_proj.weight.data *= locality_strength

    def get_rel_indices(self, num_patches):
        img_size = int(num_patches**.5)
        rel_indices   = torch.zeros(1, num_patches, num_patches, 3)
        ind = torch.arange(img_size).view(1,-1) - torch.arange(img_size).view(-1, 1)
        indx = ind.repeat(img_size,img_size)
        indy = ind.repeat_interleave(img_size,dim=0).repeat_interleave(img_size,dim=1)
        indd = indx**2 + indy**2
        rel_indices[:,:,:,2] = indd.unsqueeze(0)
        rel_indices[:,:,:,1] = indy.unsqueeze(0)
        rel_indices[:,:,:,0] = indx.unsqueeze(0)
        device = self.qk.weight.device
        self.rel_indices = rel_indices.to(device)


class MHSA(nn.Module):
    def __init__(self, dim, num_heads=8, qkv_bias=False, qk_scale=None, attn_drop=0., proj_drop=0., fc=None):
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = qk_scale or head_dim ** -0.5

        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)
        self.apply(self._init_weights)

    def reset_parameters(self):
        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    def get_attention_map(self, x, return_map = False):
        B, N, C = x.shape
        qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]
        attn_map = (q @ k.transpose(-2, -1)) * self.scale
        attn_map = attn_map.softmax(dim=-1).mean(0)

        img_size = int(N**.5)
        ind = torch.arange(img_size).view(1,-1) - torch.arange(img_size).view(-1, 1)
        indx = ind.repeat(img_size,img_size)
        indy = ind.repeat_interleave(img_size,dim=0).repeat_interleave(img_size,dim=1)
        indd = indx**2 + indy**2
        distances = indd**.5
        distances = distances.to('cuda')

        dist = torch.einsum('nm,hnm->h', (distances, attn_map))
        dist /= N

        if return_map:
            return dist, attn_map
        else:
            return dist

    def forward(self, x):
        B, N, C = x.shape
        qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]

        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)

        x = (attn @ v).transpose(1, 2).reshape(B, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x, attn


class ScaleNorm(nn.Module):
    """See
    https://github.com/lucidrains/reformer-pytorch/blob/a751fe2eb939dcdd81b736b2f67e745dc8472a09/reformer_pytorch/reformer_pytorch.py#L143
    """
    def __init__(self, dim, eps=1e-5):
        super().__init__()
        self.g = nn.Parameter(torch.ones(1))
        self.eps = eps

    def forward(self, x):
        n = torch.norm(x, dim=-1, keepdim=True).clamp(min=self.eps)
        return x / n * self.g


class Block(nn.Module):

    def __init__(self, dim, num_heads,  mlp_ratio=4., qkv_bias=False, qk_scale=None, drop=0., attn_drop=0.,
                 drop_path=0., act_layer=nn.GELU, norm_layer=nn.LayerNorm, attention_type=GPSA,
                 fc=nn.Linear, **kwargs):
        super().__init__()
        self.norm1 = norm_layer(dim)
        self.attn = attention_type(dim, num_heads=num_heads, qkv_bias=qkv_bias, qk_scale=qk_scale, attn_drop=attn_drop, proj_drop=drop, fc=fc, **kwargs)
        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()
        self.norm2 = norm_layer(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = Mlp(in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop, fc=fc)

    def reset_parameters(self):
        self.norm1.reset_parameters()
        self.norm2.reset_parameters()
        self.attn.reset_parameters()
        self.mlp.apply(self.mlp._init_weights)

    def forward(self, x, mask_heads=None, task_index=1, attn_mask=None):
     if isinstance(self.attn, (ClassAttention, JointCA)):
        cls_token = x[:, :task_index]
        xx = self.norm1(x)
        xx, attn, v = self.attn(xx, mask_heads=mask_heads, nb=task_index, attn_mask=attn_mask)
        cls_token = self.drop_path(xx[:, :task_index]) + cls_token
        cls_token = self.drop_path(self.mlp(self.norm2(cls_token))) + cls_token
        return cls_token, attn, v

     xx = self.norm1(x)
     out = self.attn(xx)

    # Safe unpack
     if isinstance(out, tuple):
        if len(out) >= 3:
            xx, attn, v = out[:3]
        elif len(out) == 2:
            xx, attn = out
            v = None
        else:
            xx = out[0]
            attn = v = None
     else:
        xx = out
        attn = v = None

     x = self.drop_path(xx) + x
     x = self.drop_path(self.mlp(self.norm2(x))) + x
     return x, attn, v



class ClassAttention(nn.Module):
    # taken from https://github.com/rwightman/pytorch-image-models/blob/master/timm/models/vision_transformer.py
    # with slight modifications to do CA
    def __init__(self, dim, num_heads=8, qkv_bias=False, qk_scale=None, attn_drop=0., proj_drop=0., fc=nn.Linear):
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = qk_scale or head_dim ** -0.5

        self.q = fc(dim, dim, bias=qkv_bias)
        self.k = fc(dim, dim, bias=qkv_bias)
        self.v = fc(dim, dim, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = fc(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

        self.apply(self._init_weights)

    def reset_parameters(self):
        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    def forward(self, x, mask_heads=None, **kwargs):
        B, N, C = x.shape
        q = self.q(x[:,0]).unsqueeze(1).reshape(B, 1, self.num_heads, C // self.num_heads).permute(0, 2, 1, 3)
        k = self.k(x).reshape(B, N, self.num_heads, C // self.num_heads).permute(0, 2, 1, 3)

        q = q * self.scale
        v = self.v(x).reshape(B, N, self.num_heads, C // self.num_heads).permute(0, 2, 1, 3)

        attn = (q @ k.transpose(-2, -1))
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)

        if mask_heads is not None:
            mask_heads = mask_heads.expand(B, self.num_heads, -1, N)
            attn = attn * mask_heads

        x_cls = (attn @ v).transpose(1, 2).reshape(B, 1, C)
        x_cls = self.proj(x_cls)
        x_cls = self.proj_drop(x_cls)

        return x_cls, attn, v


class JointCA(nn.Module):
    """Forward all task tokens together.

    It uses a masked attention so that task tokens don't interact between them.
    It should have the same results as independent forward per task token but being
    much faster.

    HOWEVER, it works a bit worse (like ~2pts less in 'all top-1' CIFAR100 50 steps).
    So if anyone knows why, please tell me!
    """
    def __init__(self, dim, num_heads=8, qkv_bias=False, qk_scale=None, attn_drop=0., proj_drop=0., fc=nn.Linear):
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = qk_scale or head_dim ** -0.5

        self.q = fc(dim, dim, bias=qkv_bias)
        self.k = fc(dim, dim, bias=qkv_bias)
        self.v = fc(dim, dim, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = fc(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

        self.apply(self._init_weights)

    def reset_parameters(self):
        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    @lru_cache(maxsize=1)
    def get_attention_mask(self, attn_shape, nb_task_tokens):
        """Mask so that task tokens don't interact together.

        Given two task tokens (t1, t2) and three patch tokens (p1, p2, p3), the
        attention matrix is:

        t1-t1 t1-t2 t1-p1 t1-p2 t1-p3
        t2-t1 t2-t2 t2-p1 t2-p2 t2-p3

        So that the mask (True values are deleted) should be:

        False True False False False
        True False False False False
        """
        mask = torch.zeros(attn_shape, dtype=torch.bool)
        for i in range(nb_task_tokens):
            mask[:, i, :i] = True
            mask[:, i, i+1:nb_task_tokens] = True
        return mask

    def forward(self, x, attn_mask=False, nb_task_tokens=1, **kwargs):
        B, N, C = x.shape
        q = self.q(x[:,:nb_task_tokens]).reshape(B, nb_task_tokens, self.num_heads, C // self.num_heads).permute(0, 2, 1, 3)
        k = self.k(x).reshape(B, N, self.num_heads, C // self.num_heads).permute(0, 2, 1, 3)

        q = q * self.scale
        v = self.v(x).reshape(B, N, self.num_heads, C // self.num_heads).permute(0, 2, 1, 3)

        attn = (q @ k.transpose(-2, -1))
        if attn_mask:
            mask = self.get_attention_mask(attn.shape, nb_task_tokens)
            attn[mask] = -float('inf')
        attn = attn.softmax(dim=-1)

        attn = self.attn_drop(attn)

        x_cls = (attn @ v).transpose(1, 2).reshape(B, nb_task_tokens, C)
        x_cls = self.proj(x_cls)
        x_cls = self.proj_drop(x_cls)

        return x_cls, attn, v


class PatchEmbed(nn.Module):
    """ Image to Patch Embedding, from timm
    """
    def __init__(self, img_size=224, patch_size=16, in_chans=3, embed_dim=768):
        super().__init__()
        img_size = to_2tuple(img_size)
        patch_size = to_2tuple(patch_size)
        num_patches = (img_size[1] // patch_size[1]) * (img_size[0] // patch_size[0])
        self.img_size = img_size
        self.patch_size = patch_size
        self.num_patches = num_patches

        self.proj = nn.Conv2d(in_chans, embed_dim, kernel_size=patch_size, stride=patch_size)
        self.apply(self._init_weights)

    def reset_parameters(self):
        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    def forward(self, x):
        B, C, H, W = x.shape
        #assert H == self.img_size[0] and W == self.img_size[1], \
        #    f"Input image size ({H}*{W}) doesn't match model ({self.img_size[0]}*{self.img_size[1]})."
        x = self.proj(x).flatten(2).transpose(1, 2)
        return x



class HybridEmbed(nn.Module):
    """ CNN Feature Map Embedding, from timm
    """
    def __init__(self, backbone, img_size=224, feature_size=None, in_chans=3, embed_dim=768):
        super().__init__()
        assert isinstance(backbone, nn.Module)
        img_size = to_2tuple(img_size)
        self.img_size = img_size
        self.backbone = backbone
        if feature_size is None:
            with torch.no_grad():
                training = backbone.training
                if training:
                    backbone.eval()
                o = self.backbone(torch.zeros(1, in_chans, img_size[0], img_size[1]))[-1]
                feature_size = o.shape[-2:]
                feature_dim = o.shape[1]
                backbone.train(training)
        else:
            feature_size = to_2tuple(feature_size)
            feature_dim = self.backbone.feature_info.channels()[-1]
        self.num_patches = feature_size[0] * feature_size[1]
        self.proj = nn.Linear(feature_dim, embed_dim)
        self.apply(self._init_weights)

    def reset_parameters(self):
        self.apply(self._init_weights)

    def forward(self, x):
        x = self.backbone(x)[-1]
        x = x.flatten(2).transpose(1, 2)
        x = self.proj(x)
        return x


class ConVit(nn.Module):
    """ Vision Transformer with support for patch or hybrid CNN input stage
    """
    def __init__(self, img_size=224, patch_size=16, in_chans=3, num_classes=1000, embed_dim=768, depth=12,
                 num_heads=12, mlp_ratio=4., qkv_bias=False, qk_scale=None, drop_rate=0., attn_drop_rate=0.,
                 drop_path_rate=0., hybrid_backbone=None, norm_layer='layer',
                 local_up_to_layer=3, locality_strength=1., use_pos_embed=True,
                 class_attention=False, ca_type='base',
        ):
        super().__init__()
        self.num_classes = num_classes
        self.num_heads = num_heads
        self.embed_dim = embed_dim
        self.local_up_to_layer = local_up_to_layer
        self.num_features = self.final_dim = self.embed_dim = embed_dim  # num_features for consistency with other models
        self.locality_strength = locality_strength
        self.use_pos_embed = use_pos_embed

        if norm_layer == 'layer':
            norm_layer = nn.LayerNorm
        elif norm_layer == 'scale':
            norm_layer = ScaleNorm
        else:
            raise NotImplementedError(f'Unknown normalization {norm_layer}')

        if hybrid_backbone is not None:
            self.patch_embed = HybridEmbed(
                hybrid_backbone, img_size=img_size, in_chans=in_chans, embed_dim=embed_dim)
        else:
            self.patch_embed = PatchEmbed(
                img_size=img_size, patch_size=patch_size, in_chans=in_chans, embed_dim=embed_dim)
        num_patches = self.patch_embed.num_patches
        self.num_patches = num_patches

        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        self.pos_drop = nn.Dropout(p=drop_rate)

        if self.use_pos_embed:
            self.pos_embed = nn.Parameter(torch.zeros(1, num_patches, embed_dim))
            trunc_normal_(self.pos_embed, std=.02)

        dpr = [x.item() for x in torch.linspace(0, drop_path_rate, depth)]  # stochastic depth decay rule

        blocks = []

        if ca_type == 'base':
            ca_block = ClassAttention
        elif ca_type == 'jointca':
            ca_block = JointCA
        else:
            raise ValueError(f'Unknown CA type {ca_type}')

        for layer_index in range(depth):
            if layer_index < local_up_to_layer:
                # Convit
                block = Block(
                    dim=embed_dim, num_heads=num_heads, mlp_ratio=mlp_ratio, qkv_bias=qkv_bias, qk_scale=qk_scale,
                    drop=drop_rate, attn_drop=attn_drop_rate, drop_path=dpr[layer_index], norm_layer=norm_layer,
                    attention_type=GPSA, locality_strength=locality_strength
                )
            elif not class_attention:
                # Convit
                block = Block(
                    dim=embed_dim, num_heads=num_heads, mlp_ratio=mlp_ratio, qkv_bias=qkv_bias, qk_scale=qk_scale,
                    drop=drop_rate, attn_drop=attn_drop_rate, drop_path=dpr[layer_index], norm_layer=norm_layer,
                    attention_type=MHSA
                )
            else:
                # CaiT
                block = Block(
                    dim=embed_dim, num_heads=num_heads, mlp_ratio=mlp_ratio, qkv_bias=qkv_bias, qk_scale=qk_scale,
                    drop=drop_rate, attn_drop=attn_drop_rate, drop_path=dpr[layer_index], norm_layer=norm_layer,
                    attention_type=ca_block
                )

            blocks.append(block)

        self.blocks = nn.ModuleList(blocks)
        self.norm = norm_layer(embed_dim)
        self.use_class_attention = class_attention

        # Classifier head
        self.feature_info = [dict(num_chs=embed_dim, reduction=0, module='head')]
        self.head = nn.Linear(embed_dim, num_classes) if num_classes > 0 else nn.Identity()

        trunc_normal_(self.cls_token, std=.02)
        self.head.apply(self._init_weights)

    def freeze(self, names):
        for name in names:
            if name == 'all':
                return cutils.freeze_parameters(self)
            elif name == 'old_heads':
                self.head.freeze(name)
            elif name == 'backbone':
                cutils.freeze_parameters(self.blocks)
                cutils.freeze_parameters(self.patch_embed)
                cutils.freeze_parameters(self.pos_embed)
                cutils.freeze_parameters(self.norm)
            else:
                raise NotImplementedError(f'Unknown name={name}.')

    def reset_classifier(self):
        self.head.apply(self._init_weights)

    def reset_parameters(self):
        for b in self.blocks:
            b.reset_parameters()
        self.norm.reset_parameters()
        self.head.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    def get_internal_losses(self, clf_loss):
        return {}

    def end_finetuning(self):
        pass

    def begin_finetuning(self):
        pass

    def epoch_log(self):
        return {}

    @torch.jit.ignore
    def no_weight_decay(self):
        return {'pos_embed', 'cls_token'}

    def get_classifier(self):
        return self.head

    def forward_sa(self, x):
        B = x.shape[0]
        x = self.patch_embed(x)

        if self.use_pos_embed:
            x = x + self.pos_embed
        x = self.pos_drop(x)

        for blk in self.blocks[:self.local_up_to_layer]:
            x, _ = blk(x)

        return x

    def forward_features(self, x, final_norm=True):
        B = x.shape[0]
        x = self.patch_embed(x)

        cls_tokens = self.cls_token.expand(B, -1, -1)

        if self.use_pos_embed:
            x = x + self.pos_embed
        x = self.pos_drop(x)

        for blk in self.blocks[:self.local_up_to_layer]:
            x, _, _ = blk(x)

        if self.use_class_attention:
            for blk in self.blocks[self.local_up_to_layer:]:
                cls_tokens, _, _ = blk(torch.cat((cls_tokens, x), dim=1))
        else:
            x = torch.cat((cls_tokens, x), dim=1)
            for blk in self.blocks[self.local_up_to_layer:]:
                x, _ , _ = blk(x)

        if final_norm:
            if self.use_class_attention:
                cls_tokens = self.norm(cls_tokens)
            else:
                x = self.norm(x)

        if self.use_class_attention:
            return cls_tokens[:, 0], None, None
        else:
            return x[:, 0], None, None

    def forward(self, x):
        x = self.forward_features(x)[0]
        x = self.head(x)
        return x





In [7]:
from timm import create_model

# 1️⃣ Load pretrained convit_base from timm
timm_convit = create_model("convit_base", pretrained=True)

# 2️⃣ Create DyTox-compatible ConViT
dytox_convit = ConVit(
    img_size=224,
    patch_size=16,
    in_chans=3,
    num_classes=2,  # binary classification
    embed_dim=768,
    depth=12,
    num_heads=16,   # matches convit_base
    mlp_ratio=4.0,
    qkv_bias=True,
    local_up_to_layer=3,
    locality_strength=1.0,
    use_pos_embed=True,
    class_attention=True,
)

# 3️⃣ Filter out classifier weights (head.*)
state_dict = timm_convit.state_dict()
filtered_dict = {k: v for k, v in state_dict.items() if not k.startswith("head.")}

# 4️⃣ Load pretrained backbone
missing, unexpected = dytox_convit.load_state_dict(filtered_dict, strict=False)

print("✅ DyTox ConViT backbone loaded from ImageNet pretrained model.")
print("Missing keys:", missing)
print("Unexpected keys:", unexpected)



✅ DyTox ConViT backbone loaded from ImageNet pretrained model.
Missing keys: ['blocks.0.attn.qk.bias', 'blocks.0.attn.v.bias', 'blocks.1.attn.qk.bias', 'blocks.1.attn.v.bias', 'blocks.2.attn.qk.bias', 'blocks.2.attn.v.bias', 'blocks.3.attn.q.weight', 'blocks.3.attn.q.bias', 'blocks.3.attn.k.weight', 'blocks.3.attn.k.bias', 'blocks.3.attn.v.bias', 'blocks.4.attn.q.weight', 'blocks.4.attn.q.bias', 'blocks.4.attn.k.weight', 'blocks.4.attn.k.bias', 'blocks.4.attn.v.bias', 'blocks.5.attn.q.weight', 'blocks.5.attn.q.bias', 'blocks.5.attn.k.weight', 'blocks.5.attn.k.bias', 'blocks.5.attn.v.bias', 'blocks.6.attn.q.weight', 'blocks.6.attn.q.bias', 'blocks.6.attn.k.weight', 'blocks.6.attn.k.bias', 'blocks.6.attn.v.bias', 'blocks.7.attn.q.weight', 'blocks.7.attn.q.bias', 'blocks.7.attn.k.weight', 'blocks.7.attn.k.bias', 'blocks.7.attn.v.bias', 'blocks.8.attn.q.weight', 'blocks.8.attn.q.bias', 'blocks.8.attn.k.weight', 'blocks.8.attn.k.bias', 'blocks.8.attn.v.bias', 'blocks.9.attn.q.weight', 'bloc

In [8]:
import copy
import torch
from timm.models.layers import trunc_normal_
from torch import nn

import continual.utils as cutils

class ContinualClassifier(nn.Module):
   
    def __init__(self, embed_dim, nb_classes):
        super().__init__()

        self.embed_dim = embed_dim
        self.nb_classes = nb_classes
        self.head = nn.Linear(embed_dim, nb_classes, bias=True)
        self.norm = nn.LayerNorm(embed_dim)

    def reset_parameters(self):
        self.head.reset_parameters()
        self.norm.reset_parameters()

    def forward(self, x):
        x = self.norm(x)
        return self.head(x)

    def add_new_outputs(self, n):
        head = nn.Linear(self.embed_dim, self.nb_classes + n, bias=True)
        head.weight.data[:-n] = self.head.weight.data

        head.to(self.head.weight.device)
        self.head = head
        self.nb_classes += n

def _get_module_device(mod):
     for p in mod.parameters():
        return p.device
    # fallback
     return torch.device("cuda" if torch.cuda.is_available() else "cpu")

class DyTox(nn.Module):
   
    def __init__(
        self,
        transformer,
        nb_classes,
        individual_classifier='',
        head_div=False,
        head_div_mode=['tr', 'ft'],
        joint_tokens=False,
        resnet=False
    ):
        super().__init__()

        self.nb_classes = nb_classes
        self.embed_dim = transformer.embed_dim
        self.individual_classifier = individual_classifier
        self.use_head_div = head_div
        self.head_div_mode = head_div_mode
        self.head_div = None
        self.joint_tokens = joint_tokens
        self.in_finetuning = False

        self.use_resnet = resnet

        self.nb_classes_per_task = [nb_classes]

        if self.use_resnet:
            print('ResNet18 backbone for ens')
            # self.backbone = resnet18()
            # self.backbone.head = nn.Sequential(
            #     nn.Conv2d(256, 384, kernel_size=1),
            #     nn.BatchNorm2d(384),
            #     nn.ReLU(inplace=True),
            #     nn.Conv2d(384, 504, kernel_size=1),
            #     nn.BatchNorm2d(504),
            #     nn.ReLU(inplace=True)
            # )
            # self.backbone.avgpool = nn.Identity()
            # self.backbone.layer4 = nn.Identity()
            # #self.backbone.layer4 = self.backbone._make_layer_nodown(
            # #    256, 512, 2, stride=1, dilation=2
            # #)
            # self.backbone = self.backbone.cuda()
            # self.backbone.embed_dim = 504
            # self.embed_dim = self.backbone.embed_dim

            # self.tabs = nn.ModuleList([
            #     Block(
            #         dim=self.embed_dim, num_heads=12, mlp_ratio=4, qkv_bias=False, qk_scale=None,
            #         drop=0., attn_drop=0., drop_path=0., norm_layer=nn.LayerNorm,
            #         attention_type=ClassAttention
            #     ).cuda()
            # ])
            # self.tabs[0].reset_parameters()

            # token = nn.Parameter(torch.zeros(1, 1, self.embed_dim).cuda())
            # trunc_normal_(token, std=.02)
            # self.task_tokens = nn.ParameterList([token])
        else:
            self.patch_embed = transformer.patch_embed
            self.pos_embed = transformer.pos_embed
            self.pos_drop = transformer.pos_drop
            self.sabs = transformer.blocks[:transformer.local_up_to_layer]
            self.tabs = transformer.blocks[transformer.local_up_to_layer:]
            self.task_tokens = nn.ParameterList([transformer.cls_token])

        if self.individual_classifier != '':
            in_dim, out_dim = self._get_ind_clf_dim()
            self.head = nn.ModuleList([
                ContinualClassifier(in_dim, out_dim).cuda()
            ])
        else:
            self.head = ContinualClassifier(
                self.embed_dim * len(self.task_tokens), sum(self.nb_classes_per_task)
            ).cuda()

    def end_finetuning(self):
        """Start FT mode, usually with backbone freezed and balanced classes."""
        self.in_finetuning = False

    def begin_finetuning(self):
        """End FT mode, usually with backbone freezed and balanced classes."""
        self.in_finetuning = True

    # def add_model(self, nb_new_classes):
    #     """Expand model as per the DyTox framework given `nb_new_classes`.

    #     :param nb_new_classes: Number of new classes brought by the new task.
    #     """
    #     self.nb_classes_per_task.append(nb_new_classes)

    #     # Class tokens ---------------------------------------------------------
    #     new_task_token = copy.deepcopy(self.task_tokens[-1])
    #     trunc_normal_(new_task_token, std=.02)
    #     self.task_tokens.append(new_task_token)
    #     # ----------------------------------------------------------------------

    #     # Diversity head -------------------------------------------------------
    #     if self.use_head_div:
    #         self.head_div = ContinualClassifier(
    #             self.embed_dim, self.nb_classes_per_task[-1] + 1
    #         ).cuda()
    #     # ----------------------------------------------------------------------

    #     # Classifier -----------------------------------------------------------
    #     if self.individual_classifier != '':
    #         in_dim, out_dim = self._get_ind_clf_dim()
    #         self.head.append(
    #             ContinualClassifier(in_dim, out_dim).cuda()
    #         )
    #     else:
    #         self.head = ContinualClassifier(
    #             self.embed_dim * len(self.task_tokens), sum(self.nb_classes_per_task)
    #         ).cuda()
    #     # ----------------------------------------------------------------------

    def _get_ind_clf_dim(self):
        """What are the input and output dim of classifier depending on its config.

        By default, DyTox is in 1-1.
        """
        if self.individual_classifier == '1-1':
            in_dim = self.embed_dim
            out_dim = self.nb_classes_per_task[-1]
        elif self.individual_classifier == '1-n':
            in_dim = self.embed_dim
            out_dim = sum(self.nb_classes_per_task)
        elif self.individual_classifier == 'n-n':
            in_dim = len(self.task_tokens) * self.embed_dim
            out_dim = sum(self.nb_classes_per_task)
        elif self.individual_classifier == 'n-1':
            in_dim = len(self.task_tokens) * self.embed_dim
            out_dim = self.nb_classes_per_task[-1]
        else:
            raise NotImplementedError(f'Unknown ind classifier {self.individual_classifier}')
        return in_dim, out_dim

    def freeze(self, names):
        """Choose what to freeze depending on the name of the module."""
        requires_grad = False
        cutils.freeze_parameters(self, requires_grad=not requires_grad)
        self.train()

        for name in names:
            if name == 'all':
                self.eval()
                return cutils.freeze_parameters(self)
            elif name == 'old_task_tokens':
                cutils.freeze_parameters(self.task_tokens[:-1], requires_grad=requires_grad)
            elif name == 'task_tokens':
                cutils.freeze_parameters(self.task_tokens, requires_grad=requires_grad)
            elif name == 'sab':
                if self.use_resnet:
                    self.backbone.eval()
                    cutils.freeze_parameters(self.backbone, requires_grad=requires_grad)
                else:
                    self.sabs.eval()
                    cutils.freeze_parameters(self.patch_embed, requires_grad=requires_grad)
                    cutils.freeze_parameters(self.pos_embed, requires_grad=requires_grad)
                    cutils.freeze_parameters(self.sabs, requires_grad=requires_grad)
            elif name == 'tab':
                self.tabs.eval()
                cutils.freeze_parameters(self.tabs, requires_grad=requires_grad)
            elif name == 'old_heads':
                self.head[:-1].eval()
                cutils.freeze_parameters(self.head[:-1], requires_grad=requires_grad)
            elif name == 'heads':
                self.head.eval()
                cutils.freeze_parameters(self.head, requires_grad=requires_grad)
            elif name == 'head_div':
                self.head_div.eval()
                cutils.freeze_parameters(self.head_div, requires_grad=requires_grad)
            else:
                raise NotImplementedError(f'Unknown name={name}.')

    def param_groups(self):
        return {
            'all': self.parameters(),
            'old_task_tokens': self.task_tokens[:-1],
            'task_tokens': self.task_tokens.parameters(),
            'new_task_tokens': [self.task_tokens[-1]],
            'sa': self.sabs.parameters(),
            'patch': self.patch_embed.parameters(),
            'pos': [self.pos_embed],
            'ca': self.tabs.parameters(),
            'old_heads': self.head[:-self.nb_classes_per_task[-1]].parameters() \
                              if self.individual_classifier else \
                              self.head.parameters(),
            'new_head': self.head[-1].parameters() if self.individual_classifier else self.head.parameters(),
            'head': self.head.parameters(),
            'head_div': self.head_div.parameters() if self.head_div is not None else None
        }

    def reset_classifier(self):
        if isinstance(self.head, nn.ModuleList):
            for head in self.head:
                head.reset_parameters()
        else:
            self.head.reset_parameters()

    def hook_before_update(self):
        pass

    def hook_after_update(self):
        pass

    def hook_after_epoch(self):
        pass

    def epoch_log(self):
        """Write here whatever you want to log on the internal state of the model."""
        log = {}

        # Compute mean distance between class tokens
        mean_dist, min_dist, max_dist = [], float('inf'), 0.
        with torch.no_grad():
            for i in range(len(self.task_tokens)):
                for j in range(i + 1, len(self.task_tokens)):
                    dist = torch.norm(self.task_tokens[i] - self.task_tokens[j], p=2).item()
                    mean_dist.append(dist)

                    min_dist = min(dist, min_dist)
                    max_dist = max(dist, max_dist)

        if len(mean_dist) > 0:
            mean_dist = sum(mean_dist) / len(mean_dist)
        else:
            mean_dist = 0.
            min_dist = 0.

        assert min_dist <= mean_dist <= max_dist, (min_dist, mean_dist, max_dist)
        log['token_mean_dist'] = round(mean_dist, 5)
        log['token_min_dist'] = round(min_dist, 5)
        log['token_max_dist'] = round(max_dist, 5)
        return log

    def get_internal_losses(self, clf_loss):
        """If you want to compute some internal loss, like a EWC loss for example.

        :param clf_loss: The main classification loss (if you wanted to use its gradient for example).
        :return: a dictionnary of losses, all values will be summed in the final loss.
        """
        int_losses = {}
        return int_losses

    def forward_features(self, x):
        # Shared part, this is the ENCODER
        B = x.shape[0]

        if self.use_resnet:
            x, self.feats = self.backbone.forward_tokens(x)
        else:
            x = self.patch_embed(x)
            x = x + self.pos_embed
            x = self.pos_drop(x)

            self.feats = []
            for blk in self.sabs:
                x, attn, v = blk(x)
                self.feats.append(x)
            self.feats.pop(-1)

        # Specific part, this is what we called the "task specific DECODER"
        if self.joint_tokens:
            return self.forward_features_jointtokens(x)

        tokens = []
        attentions = []
        mask_heads = None

        for task_token in self.task_tokens:
            task_token = task_token.expand(B, -1, -1)

            for blk in self.tabs:
                task_token, attn, v = blk(torch.cat((task_token, x), dim=1), mask_heads=mask_heads)

            attentions.append(attn)
            tokens.append(task_token[:, 0])

        self._class_tokens = tokens
        return tokens, tokens[-1], attentions

    def forward_features_jointtokens(self, x):
        """Method to do a single TAB forward with all task tokens.

        A masking is used to avoid interaction between tasks. In theory it should
        give the same results as multiple TAB forward, but in practice it's a little
        bit worse, not sure why. So if you have an idea, please tell me!
        """
        B = len(x)

        task_tokens = torch.cat(
            [task_token.expand(B, 1, -1) for task_token in self.task_tokens],
            dim=1
        )

        for blk in self.tabs:
            task_tokens, _, _ = blk(
                torch.cat((task_tokens, x), dim=1),
                task_index=len(self.task_tokens),
                attn_mask=True
            )

        if self.individual_classifier in ('1-1', '1-n'):
            return task_tokens.permute(1, 0, 2), task_tokens[:, -1], None
        return task_tokens.view(B, -1), task_tokens[:, -1], None


# -------------------------
# Helper: get current device
# -------------------------


# -------------------------
# New add_model implementation
# -------------------------
    def add_model(self, nb_new_classes):
     """Expand model safely (device-aware)."""
    # record new classes
     self.nb_classes_per_task.append(int(nb_new_classes))

    # device to place newly created params
     device = _get_module_device(self)

    # --- new class token ---
     new_task_token = copy.deepcopy(self.task_tokens[-1])
     trunc_normal_(new_task_token, std=.02)
    # place on device
     new_task_token = new_task_token.to(device)
     self.task_tokens.append(new_task_token)
    # ------------------------

    # --- diversity head (optional) ---
     if self.use_head_div:
        self.head_div = ContinualClassifier(self.embed_dim, self.nb_classes_per_task[-1] + 1)
        self.head_div.to(device)
        self.head_div.reset_parameters()
    # ---------------------------------

    # --- classifier expansion ---
     if self.individual_classifier != '':
        # individual heads per task (1-1 typical)
        in_dim, out_dim = self._get_ind_clf_dim()
        new_head = ContinualClassifier(in_dim, out_dim)
        new_head.to(device)
        new_head.reset_parameters()
        # append to ModuleList
        if not isinstance(self.head, nn.ModuleList):
            # convert single head -> ModuleList (edge-case)
            old = self.head
            self.head = nn.ModuleList([old])
        self.head.append(new_head)
     else:
        # single-head case: expand output dim
        # create a new head with expanded outputs and copy weights
        total_out = sum(self.nb_classes_per_task)
        new_head = ContinualClassifier(self.embed_dim * len(self.task_tokens), total_out)
        new_head.to(device)
        # If previous head exists, try to copy weights for the shared dims
        try:
            # operate on cpu temporarily to avoid device issues
            old_w = self.head.head.weight.data.clone()
            old_b = self.head.head.bias.data.clone()
            new_head.head.weight.data[:old_w.shape[0]] = old_w
            new_head.head.bias.data[:old_b.shape[0]] = old_b
        except Exception:
            # ignore if incompatible, reset instead
            new_head.reset_parameters()
        self.head = new_head
    # -----------------------------

# -------------------------
# Safe forward_classifier
# -------------------------
    def forward_classifier(self, tokens, last_token):
     """
     tokens: list of per-task embeddings (each shape [B, embed_dim])
     last_token: tokens[-1]
     Returns dict{'logits','div','tokens'} (logits on correct device)
     """
     device = tokens[0].device if isinstance(tokens, (list, tuple)) and len(tokens) > 0 else _get_module_device(self)
     logits_div = None

     if self.individual_classifier != '':
        # Expect self.head to be ModuleList with one head per task
        logits_list = []
        for i, head in enumerate(self.head):
            if self.individual_classifier in ('1-n', '1-1'):
                logits_list.append(head(tokens[i]))  # each -> [B, num_classes_i]
            else:
                # n-1 or n-n: concatenate tokens up to i
                concat = torch.cat(tokens[:i+1], dim=1)
                logits_list.append(head(concat))

        if self.individual_classifier in ('1-1', 'n-1'):
            logits = torch.cat(logits_list, dim=1)
        else:
            # combine into shared final logits (original repo does some averaging)
            final_logits = torch.zeros_like(logits_list[-1], device=device)
            for i, l in enumerate(logits_list):
                final_logits[:, :l.shape[1]] += l
            for i, c in enumerate(self.nb_classes_per_task):
                final_logits[:, :c] /= (len(self.nb_classes_per_task) - i)
            logits = final_logits
     elif isinstance(tokens, torch.Tensor):
        logits = self.head(tokens)
     else:
        # single-head with tokens list: concatenate
        concat = torch.cat(tokens, dim=1)
        logits = self.head(concat)

    # head div if present
     if self.head_div is not None and eval_training_finetuning(self.head_div_mode, self.in_finetuning):
        logits_div = self.head_div(last_token)

     return {'logits': logits, 'div': logits_div, 'tokens': tokens}

# -------------------------
# set_task: mark current task
# -------------------------
    def set_task(self, task_id: int):
    # simple setter used at train/eval time
     self.current_task_id = int(task_id)

# -------------------------
# Safe forward: prefer current_task if set (1-1 typical)
# -------------------------
    def forward(self, x):
     tokens, last_token, _ = self.forward_features(x)
     if hasattr(self, "current_task_id") and self.individual_classifier in ('1-1', '1-n'):
        # use only the active head (prevents accidentally scoring other heads)
        idx = int(self.current_task_id)
        logits = self.head[idx](tokens[idx])
        logits_div = None
        if self.head_div is not None and eval_training_finetuning(self.head_div_mode, self.in_finetuning):
            logits_div = self.head_div(last_token)
        return {'logits': logits, 'div': logits_div, 'tokens': tokens}
    # fallback to full classifier behavior
     return self.forward_classifier(tokens, last_token)




def eval_training_finetuning(mode, in_ft):
    if 'tr' in mode and 'ft' in mode:
        return True
    if 'tr' in mode and not in_ft:
        return True
    if 'ft' in mode and in_ft:
        return True
    return False

In [9]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = DyTox(
    transformer=dytox_convit,
    nb_classes=2,                   # 🔥 Important
    individual_classifier='1-1',
    head_div=False,
    head_div_mode=['tr', 'ft'],
    joint_tokens=False,
    resnet=False
).to(device)

In [10]:
class LabelSmoothingCrossEntropyBoosting(nn.Module):
    """
    NLL loss with label smoothing.
    """
    def __init__(self, smoothing=0.1, alpha=1, gamma=1):
        """
        Constructor for the LabelSmoothing module.
        :param smoothing: label smoothing factor
        """
        super().__init__()
        assert smoothing < 1.0
        self.smoothing = smoothing
        self.confidence = 1. - smoothing

        self.alpha = alpha
        self.gamma = gamma

    def forward(self, x, target, boosting_output=None, boosting_focal=None):
        if boosting_output is None:
            return self._base_loss(x, target)
        return self._focal_loss(x, target, boosting_output, boosting_focal)

    def _focal_loss(self, x, target, boosting_output, boosting_focal):
        logprobs = F.log_softmax(x, dim=-1)

        if boosting_focal == 'old':
            pt = boosting_output.softmax(-1)[..., :-1]

            f = torch.ones_like(logprobs)
            f[:, :boosting_output.shape[1] - 1] = self.alpha * (1 - pt) ** self.gamma
            logprobs = f * logprobs
        elif boosting_focal == 'new':
            pt = boosting_output.softmax(-1)[..., -1]
            nb_old_classes = boosting_output.shape[1] - 1

            f = torch.ones_like(logprobs)
            f[:, nb_old_classes:] = self.alpha * (1 - pt[:, None]) ** self.gamma
            logprobs = f * logprobs
        else:
            assert False, (boosting_focal)

        nll_loss = -logprobs.gather(dim=-1, index=target.unsqueeze(1))
        nll_loss = nll_loss.squeeze(1)
        smooth_loss = -logprobs.mean(dim=-1)
        loss = self.confidence * nll_loss + self.smoothing * smooth_loss
        return loss.mean()

    def _base_loss(self, x, target):
        logprobs = F.log_softmax(x, dim=-1)
        nll_loss = -logprobs.gather(dim=-1, index=target.unsqueeze(1))
        nll_loss = nll_loss.squeeze(1)
        smooth_loss = -logprobs.mean(dim=-1)
        loss = self.confidence * nll_loss + self.smoothing * smooth_loss
        return loss.mean()


class SoftTargetCrossEntropyBoosting(nn.Module):

    def __init__(self, alpha=1, gamma=1):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, x, target, boosting_output=None, boosting_focal=None):
        if boosting_output is None:
            return torch.sum(-target * F.log_softmax(x, dim=-1), dim=-1).mean()

        if boosting_focal == 'old':
            pt = boosting_output.softmax(-1)[..., :-1]

            f = torch.ones_like(x)
            f[:, :boosting_output.shape[1] - 1] = self.alpha * (1 - pt) ** self.gamma
        elif boosting_focal == 'new':
            pt = boosting_output.softmax(-1)[..., -1]

            nb_old_classes = boosting_output.shape[1] - 1

            f = torch.ones_like(x)
            f[:, nb_old_classes:] = self.alpha * (1 - pt[:, None]) ** self.gamma
        else:
            assert False, (boosting_focal)

        return torch.sum(-target * f * F.log_softmax(x, dim=-1), dim=-1).mean()

In [11]:
import torch
import torch.nn.functional as F
from torch import nn

class DistillationLoss(nn.Module):
    """
    Hybrid version — identical math, but with safe device handling & small memory optimizations.
    """
    def __init__(self, base_criterion: nn.Module, teacher_model: nn.Module,
                 distillation_type: str = 'soft', alpha: float = 0.5, tau: float = 2.0):
        super().__init__()
        assert distillation_type in ['none', 'soft', 'hard']
        self.base_criterion = base_criterion
        self.teacher_model = teacher_model
        self.distillation_type = distillation_type
        self.alpha = float(alpha)
        self.tau = float(tau)

    def _extract_logits(self, out):
        """Normalizes many output formats to (main_logits, kd_logits or None)."""
        if isinstance(out, torch.Tensor):
            return out, None
        if isinstance(out, (tuple, list)):
            return out[0], out[1] if len(out) > 1 else None
        if isinstance(out, dict):
            main = None
            kd = None
            for k in ['logits', 'out', 'pred', 'logit']:
                if k in out:
                    main = out[k]
                    break
            for k in ['kd', 'kd_logits', 'distill', 'distill_logits', 'logits_kd']:
                if k in out:
                    kd = out[k]
                    break
            if main is None:
                # fallback: pick first tensor in dict
                for v in out.values():
                    if isinstance(v, torch.Tensor):
                        main = v
                        break
            return main, kd
        raise ValueError("Unsupported model output type for distillation")

    def forward(self, inputs, outputs, labels):
        """Compute classification + distillation loss safely."""
        # Determine device
        device = None
        try:
            params = list(self.base_criterion.parameters())
            if len(params) > 0:
                device = params[0].device
        except Exception:
            pass
        if device is None:
            # Fallback: infer from outputs or labels
            if isinstance(outputs, torch.Tensor):
                device = outputs.device
            elif isinstance(labels, torch.Tensor):
                device = labels.device
            else:
                device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Move teacher model to same device if not already
        if self.teacher_model is not None:
            self.teacher_model.to(device)

        # Extract logits
        student_logits, student_kd_logits = self._extract_logits(outputs)
        student_logits = student_logits.to(device)
        labels = labels.to(device)
        inputs = inputs.to(device)

        base_loss = self.base_criterion(student_logits, labels)

        # If no distillation, return base loss
        if self.distillation_type == 'none' or self.teacher_model is None:
            return base_loss

        # Teacher forward (no grad)
        with torch.no_grad():
            teacher_out = self.teacher_model(inputs)
            teacher_logits, _ = self._extract_logits(teacher_out)
            teacher_logits = teacher_logits.to(device)

        if student_kd_logits is None:
            student_kd_logits = student_logits

        # Distillation loss
        if self.distillation_type == 'soft':
            T = float(self.tau)
            s_log_prob = F.log_softmax(student_kd_logits / T, dim=1)
            t_prob = F.softmax(teacher_logits / T, dim=1)
            distillation_loss = F.kl_div(s_log_prob, t_prob, reduction='batchmean') * (T * T)
        else:  # hard
            teacher_pred = torch.argmax(teacher_logits, dim=1)
            distillation_loss = F.cross_entropy(student_kd_logits, teacher_pred)

        # Combine losses
        loss = base_loss * (1.0 - self.alpha) + distillation_loss * self.alpha
        return loss


In [13]:
import torch
import copy

def deepcopy_model_to_cpu(model): """ Safe version that clones the model's state_dict (not tensors with grad_fn) Works even if model uses weight norm, buffers, or task tokens. """ # Ensure everything is detached and gradients cleared for p in model.parameters(): if p.grad is not None: p.grad = None p.detach_() torch.cuda.empty_cache() # Create a new instance of same class teacher = model.__class__.__new__(model.__class__) # Shallow copy static attributes teacher.__dict__ = copy.copy(model.__dict__) # Safely copy weights with torch.no_grad(): state_dict = { k: v.detach().cpu().clone() for k, v in model.state_dict().items() } teacher.load_state_dict(state_dict, strict=False) # Freeze params and move to CPU for p in teacher.parameters(): p.requires_grad = False teacher.eval() teacher.cpu() return teacher


In [15]:
# -----------------------------
@torch.no_grad()
def evaluate_task(model, loader, device):
    model.eval()
    total, correct = 0, 0
    for imgs, labels in loader:
        imgs, labels = imgs.to(device), labels.to(device)
        
        out = model(imgs)
        logits = out["logits"] if isinstance(out, dict) else out
        preds = torch.argmax(logits, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    return 100.0 * correct / total if total > 0 else 0.0


# -----------------------------
# Compute average accuracy and forgetting
# -----------------------------
def compute_metrics(acc_matrix):
    n_tasks = len(acc_matrix)
    final_acc = acc_matrix[-1]
    avg_acc = sum(final_acc) / len(final_acc)

    forgetting = []
    # Compute forgetting for all previous tasks up to current
    for i in range(n_tasks - 1):
        # accuracy of task i after each later task
        acc_i_over_time = [acc_matrix[t][i] for t in range(i, n_tasks)]
        max_prev = max(acc_i_over_time[:-1])  # best accuracy before last
        final = acc_i_over_time[-1]
        forgetting.append(max_prev - final)

    avg_forgetting = sum(forgetting) / len(forgetting) if forgetting else 0.0
    return avg_acc, avg_forgetting

In [16]:
import torch
import torch.nn.functional as F
import random


import torch, random
from torch.utils.data import TensorDataset, DataLoader

class ReplayBuffer:
    def __init__(self, capacity_per_task=200, store_size=(112,112)):
        self.capacity_per_task = capacity_per_task
        self.store_size = store_size
        self.data = []       # list of (C,H,W) tensors (CPU)
        self.targets = []    # list of int labels
        self.task_ids = []   # list of int task ids

    def __len__(self):
        return len(self.data)

    def is_empty(self):
        return len(self.data) == 0

    def add_samples(self, task_id, loader, max_samples=None):
        """Collect up to max_samples exemplars from the given loader (stored on CPU)."""
        if max_samples is None:
            max_samples = self.capacity_per_task
        collected = 0
        for imgs, labels in loader:
            for i in range(imgs.size(0)):
                if collected >= max_samples:
                    break
                img = imgs[i].cpu()
                lab = int(labels[i].item())
                self.data.append(img)
                self.targets.append(lab)
                self.task_ids.append(int(task_id))
                collected += 1
            if collected >= max_samples:
                break

        # --- Enforce per-task capacity ---
        items = list(zip(self.data, self.targets, self.task_ids))
        kept = []
        for tid in sorted(set(self.task_ids)):
            per_task = [x for x in items if x[2] == tid]
            if len(per_task) > self.capacity_per_task:
                per_task = per_task[-self.capacity_per_task:]
            kept.extend(per_task)
        self.data = [x[0] for x in kept]
        self.targets = [x[1] for x in kept]
        self.task_ids = [x[2] for x in kept]

    def sample_batch(self, batch_size=16, device="cuda"):
        """Randomly sample a batch (returns tensors on device)."""
        if len(self.data) == 0:
            return None, None, None
        bs = min(batch_size, len(self.data))
        idx = random.sample(range(len(self.data)), bs)
        imgs = torch.stack([self.data[i] for i in idx]).to(device)
        labs = torch.tensor([self.targets[i] for i in idx], dtype=torch.long, device=device)
        tasks = torch.tensor([self.task_ids[i] for i in idx], dtype=torch.long, device=device)
        return imgs, labs, tasks

    def get_loader(self, batch_size=32, shuffle=True):
        """Return a DataLoader over all replay data (on CPU)."""
        if len(self.data) == 0:
            return None
        imgs = torch.stack(self.data)
        labs = torch.tensor(self.targets, dtype=torch.long)
        dataset = TensorDataset(imgs, labs)
        return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)


In [18]:
# === REQUIRED imports ===
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.cuda.amp import GradScaler, autocast
from torch.utils.data import DataLoader, Dataset, ConcatDataset
from tqdm import tqdm
import copy, gc, math

# === DEFAULT HYPERPARAMS (good starting point for Kaggle T4) ===
CONFIG = {
    "epochs_per_task": 10,         # main training epochs per task (increase if you want more current-task accuracy)
    "finetune_epochs": 3,        # balanced finetuning after main train
    "batch_size": 32,
    "replay_per_task": 300,      # store-size (on CPU) per task
    "replay_sample_ratio": 0.25, # fraction of minibatch to replace with replay
    "backbone_lr": 3e-5,
    "head_lr": 1e-4,
    "weight_decay": 1e-4,
    "grad_clip": 1.0,
    "unfreeze_last_sabs": 1,     # number of last shared blocks to unfreeze (ConViT)
    "distill_tau": 2.0,
    "distill_alpha": 0.5
}

# === Helper: build optimizer param groups (backbone vs head) ===
def build_optimizer_for_model(model, lr_backbone, lr_head, weight_decay):
    backbone_params = []
    head_params = []
    # Heuristic: params under 'head' name considered head; else backbone
    for name, p in model.named_parameters():
        if not p.requires_grad:
            continue
        if "head" in name:
            head_params.append(p)
        else:
            backbone_params.append(p)
    # fallback: if head empty, attempt to use last head module
    if len(head_params) == 0:
        try:
            if isinstance(model.head, nn.ModuleList):
                for p in model.head[-1].parameters():
                    if p.requires_grad:
                        head_params.append(p)
            elif isinstance(model.head, nn.Module):
                for p in model.head.parameters():
                    if p.requires_grad:
                        head_params.append(p)
        except Exception:
            pass

    param_groups = [
        {"params": backbone_params, "lr": lr_backbone, "weight_decay": weight_decay},
        {"params": head_params, "lr": lr_head, "weight_decay": weight_decay},
    ]
    return optim.AdamW(param_groups)

# === Balanced finetune dataset wrapper (uses memory + recent task data, balances classes) ===
class MemoryBalancedDataset(Dataset):
    """
    Expects memory_data: lists (imgs_cpu tensors, label ints, task_id ints)
    and new_data: a dataset (torch Dataset) for current task. We'll sample per-class balanced
    by simple upsampling/replication to equalize counts.
    """
    def __init__(self, memory_imgs, memory_labels, new_loader, transform=None, oversample_old=1):
        self.data = []
        self.labels = []
        # memory entries already CPU tensors
        if memory_imgs is not None and len(memory_imgs) > 0:
            self.data.extend(memory_imgs)
            self.labels.extend(memory_labels)
        # also add current task data (we'll pull images from loader)
        new_imgs, new_labels = [], []
        for imgs, labs in new_loader:
            for i in range(imgs.size(0)):
                new_imgs.append(imgs[i].cpu())
                new_labels.append(int(labs[i].item()))
        self.data.extend(new_imgs)
        self.labels.extend(new_labels)
        # simple balancing: replicate underrepresented classes up to max_count
        counts = {}
        for l in self.labels:
            counts[l] = counts.get(l, 0) + 1
        maxc = max(counts.values()) if counts else 0
        balanced_data, balanced_labels = [], []
        per_class = {}
        for d, l in zip(self.data, self.labels):
            per_class.setdefault(l, []).append(d)
        for cls, items in per_class.items():
            # replicate items to reach maxc
            repeat = math.ceil(maxc / max(1, len(items)))
            extended = (items * repeat)[:maxc]
            balanced_data.extend(extended)
            balanced_labels.extend([cls] * len(extended))
        self.data, self.labels = balanced_data, balanced_labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# === Main training function per-task ===
def train_task_with_replay_and_distill(
    model,
    t_id,
    train_loader,
    val_loaders,
    replay,                 # your replay object with add_samples(task_id, loader, max_samples) and sample_batch(batch_size)-> imgs_cpu, labels_cpu, tasks_cpu (or None for tasks)
    old_model_cpu,          # snapshot returned by deepcopy_model_to_cpu from previous iteration (or None)
    compute_metrics=None,
    cfg=CONFIG,
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
):
    """
    Trains model on one new task t_id in-place. Returns snapshot (old_model cpu) for next iteration and accs.
    Key assumptions:
      - model.add_model(nb_new_classes) exists; DyTox-like model returns dict {'logits':..., 'tokens':...} from forward
      - model.set_task(task_id) sets the active head/token for forward routing
      - replay has add_samples(task_id, loader, max_samples) and sample_batch(batch_size) -> (imgs_cpu, labs_cpu, tasks_cpu)
    """
    # ---- 1. prepare teacher (old snapshot) and model expansion ----
    teacher = None
    if old_model_cpu is not None:
        teacher = old_model_cpu  # CPU snapshot
        # We'll move teacher to device temporarily during forward (no grads)
    # For first task: assume model already created with first head; else call add_model appropriately
    if t_id == 0:
        model.set_task(0)
    else:
        model.add_model(2)     # CDDB adds 2 classes per task
        model.set_task(t_id)

        # Freeze old tokens & optionally old heads to protect them
        for i in range(len(model.task_tokens) - 1):
            model.task_tokens[i].requires_grad = False
        if isinstance(model.head, nn.ModuleList):
            for head_i in range(len(model.head) - 1):
                for p in model.head[head_i].parameters():
                    p.requires_grad = False

    # ---- selective unfreeze (ConViT specifics) ----
    # Freeze everything then selectively unfreeze:
    for p in model.parameters():
        p.requires_grad = False
    # unfreeze last few sabs (shared encoder blocks)
    num_unfreeze = cfg["unfreeze_last_sabs"]
    if hasattr(model, "sabs") and len(model.sabs) > 0:
        for blk in model.sabs[-num_unfreeze:]:
            for p in blk.parameters():
                p.requires_grad = True
    # unfreeze TABs (task attention blocks)
    if hasattr(model, "tabs"):
        for blk in model.tabs:
            for p in blk.parameters():
                p.requires_grad = True
    # unfreeze all head params (especially new head)
    if isinstance(model.head, nn.ModuleList):
        for head in model.head:
            for p in head.parameters():
                p.requires_grad = True
    else:
        for p in model.head.parameters():
            p.requires_grad = True
    # unfreeze last task token(s)
    for p in model.task_tokens[-1:]:
        p.requires_grad = True

    # ---- criterion (base + distillation if teacher exists) ----
    base_loss = LabelSmoothingCrossEntropyBoosting(smoothing=0.1).to(device)
    if teacher is not None:
        criterion = DistillationLoss(base_criterion=base_loss, teacher_model=None,
                                     distillation_type='soft',
                                     alpha=cfg["distill_alpha"], tau=cfg["distill_tau"])
        # We'll set teacher reference per-batch (move to device for forward only)
    else:
        criterion = base_loss

    # ---- optimizer AFTER deciding requires_grad ----
    optimizer = build_optimizer_for_model(model, cfg["backbone_lr"], cfg["head_lr"], cfg["weight_decay"])
    scaler = GradScaler()
    model.to(device).float()
    model.train()

    # ---- training epochs (main loop) ------------------------------------
    for epoch in range(cfg["epochs_per_task"]):
        running_loss = 0.0
        correct = 0
        total = 0
        pbar = tqdm(train_loader, desc=f"Task {t_id} | Epoch {epoch+1}/{cfg['epochs_per_task']}")
        for imgs, labels in pbar:
            imgs = imgs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad(set_to_none=True)

            # ensure model is routing to current task for current batch
            model.set_task(t_id)

            # --- forward current batch ---
            with autocast():
                outputs = model(imgs)
                if isinstance(criterion, DistillationLoss):
                    # if teacher exists, run teacher forward on CPU -> move to device only for logits
                    if teacher is not None:
                        # teacher is CPU; move to device temporarily for fast forward
                       if teacher is not None:
                        teacher_dev = copy.deepcopy(model).to(device)
                        teacher_dev.load_state_dict(teacher.state_dict())
                        teacher_dev.eval()
                        with torch.no_grad():
                            teacher_out = teacher_dev(imgs)
                        # free teacher_dev promptly
                        del teacher_dev
                        torch.cuda.empty_cache()
                        loss_current = criterion(imgs, outputs, labels)  # criterion will do teacher forward if you set teacher_model
                        # To be safe, set teacher inside criterion here:
                        criterion.teacher_model = None  # ensure criterion not using old internal model
                        # We will compute distillation loss manually below using teacher_out to avoid double forward
                        # Use criterion logic: extract student logits, teacher logits and compute
                        # Simpler: call a helper distill computation (we'll do a simple soft KL here)
                        student_logits, student_kd = criterion._extract_logits(outputs)
                        teacher_logits, _ = criterion._extract_logits(teacher_out)
                        if student_kd is None:
                            student_kd = student_logits
                        T = float(criterion.tau)
                        s_log_prob = F.log_softmax(student_kd / T, dim=1)
                        t_prob = F.softmax(teacher_logits / T, dim=1)
                        distill_loss = F.kl_div(s_log_prob, t_prob, reduction='batchmean') * (T * T)
                        base_l = base_loss(student_logits, labels)
                        loss_current = base_l * (1.0 - criterion.alpha) + distill_loss * criterion.alpha
                    else:
                        # no teacher -> just base loss via the wrapper
                        loss_current = criterion(imgs, outputs, labels)
                else:
                    logits = outputs["logits"] if isinstance(outputs, dict) else outputs
                    loss_current = criterion(logits, labels)

            scaler.scale(loss_current).backward()

            # --- replay step: sample a small replay batch and backprop separately per-task ---
            if t_id > 0 and len(replay.data) > 0:
                # number from config: either ratio*batch or at least 1
                replay_n = max(1, int(imgs.size(0) * cfg["replay_sample_ratio"]))
                r_imgs_cpu, r_labels_cpu, r_tasks_cpu = replay.sample_batch(replay_n)  # returns CPU tensors + task ids
                if r_imgs_cpu is not None:
                    # group by tasks to route to correct head
                    tasks_list = r_tasks_cpu.tolist() if r_tasks_cpu is not None else [0]*r_imgs_cpu.size(0)
                    # move to device and resize
                    r_imgs_full = F.interpolate(r_imgs_cpu.to(device), size=(224,224))
                    r_labels_full = r_labels_cpu.to(device)
                    # per-task grouping
                    for task_idx in sorted(set(tasks_list)):
                        idxs = [i for i, t in enumerate(tasks_list) if t == task_idx]
                        if len(idxs) == 0:
                            continue
                        sel_imgs = r_imgs_full[idxs]
                        sel_labels = r_labels_full[idxs]
                        model.set_task(int(task_idx))
                        with autocast():
                            out_replay = model(sel_imgs)
                            if isinstance(criterion, DistillationLoss) and teacher is not None:
                                # compute distill using teacher snapshot for those replay inputs
                                teacher_dev = copy.deepcopy(teacher).to(device)
                                teacher_dev.eval()
                                with torch.no_grad():
                                    teacher_out_replay = teacher_dev(sel_imgs)
                                # compute same as above:
                                student_logits_r, student_kd_r = criterion._extract_logits(out_replay)
                                teacher_logits_r, _ = criterion._extract_logits(teacher_out_replay)
                                if student_kd_r is None:
                                    student_kd_r = student_logits_r
                                T = float(criterion.tau)
                                s_log_prob = F.log_softmax(student_kd_r / T, dim=1)
                                t_prob = F.softmax(teacher_logits_r / T, dim=1)
                                distill_loss_r = F.kl_div(s_log_prob, t_prob, reduction='batchmean') * (T * T)
                                base_r = base_loss(student_logits_r, sel_labels)
                                loss_replay = base_r * (1.0 - criterion.alpha) + distill_loss_r * criterion.alpha
                                del teacher_dev; torch.cuda.empty_cache()
                            else:
                                logits_replay = out_replay["logits"] if isinstance(out_replay, dict) else out_replay
                                loss_replay = base_loss(logits_replay, sel_labels)
                        scaler.scale(loss_replay).backward()

            # ---- finalize step: unscale, clip, detect inf/nan grads, step ----
            scaler.unscale_(optimizer)
            # detect non-finite grads
            grads_ok = True
            for group in optimizer.param_groups:
                for p in group["params"]:
                    if p.grad is None:
                        continue
                    if not torch.isfinite(p.grad).all():
                        grads_ok = False
                        break
                if not grads_ok:
                    break
            if not grads_ok:
                optimizer.zero_grad(set_to_none=True)
                scaler.update()
                # skip step to avoid corrupting weights
                continue

            # clip and step
            torch.nn.utils.clip_grad_norm_([p for g in optimizer.param_groups for p in g["params"] if p.grad is not None],
                                           cfg["grad_clip"])
            scaler.step(optimizer)
            scaler.update()

            # bookkeeping
            running_loss += loss_current.item() * imgs.size(0)
            logits_out = outputs["logits"] if isinstance(outputs, dict) else outputs
            preds = torch.argmax(logits_out, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            pbar.set_postfix({"Acc": f"{100*correct/total:.2f}%", "Loss": f"{running_loss/(total+1e-9):.4f}"})

        # epoch end
        epoch_loss = running_loss / max(1, total)
        epoch_acc = 100.0 * correct / max(1, total)
        print(f"Task {t_id} | Epoch {epoch+1}/{cfg['epochs_per_task']} | Loss={epoch_loss:.4f} | Acc={epoch_acc:.2f}%")

    # ---- Balanced finetuning using memory + current data (helps both current accuracy and reduce forgetting) ----
    # Build balanced dataset
    memory_imgs = replay.data if hasattr(replay, "data") else None
    memory_labels = replay.targets if hasattr(replay, "targets") else None
    finetune_dataset = MemoryBalancedDataset(memory_imgs, memory_labels, train_loader)
    if len(finetune_dataset) > 0:
        ft_loader = DataLoader(finetune_dataset, batch_size=cfg["batch_size"], shuffle=True)
        # fine-tune only heads + last shared block (lower LR)
        # freeze everything, then unfreeze head & last sabs and tabs
        for p in model.parameters():
            p.requires_grad = False
        if hasattr(model, "sabs") and len(model.sabs) > 0:
            for p in model.sabs[-cfg["unfreeze_last_sabs"]:].parameters():
                p.requires_grad = True
        if hasattr(model, "tabs"):
            for p in model.tabs.parameters():
                p.requires_grad = True
        # unfreeze heads
        if isinstance(model.head, nn.ModuleList):
            for p in model.head[-1].parameters():  # prefer new head
                p.requires_grad = True
        else:
            for p in model.head.parameters():
                p.requires_grad = True
        # optimizer for finetune (head-only + last block)
        ft_optimizer = build_optimizer_for_model(model, lr_backbone=cfg["backbone_lr"]/10, lr_head=cfg["head_lr"], weight_decay=cfg["weight_decay"])
        ft_scaler = GradScaler()
        model.train()
        for _ in range(cfg["finetune_epochs"]):
            for imgs_ft, labels_ft in ft_loader:
                imgs_ft = imgs_ft.to(device)
                labels_ft = labels_ft.to(device)
                ft_optimizer.zero_grad(set_to_none=True)
                model.set_task(t_id)
                with autocast():
                    out_ft = model(imgs_ft)
                    logits_ft = out_ft["logits"] if isinstance(out_ft, dict) else out_ft
                    loss_ft = base_loss(logits_ft, labels_ft)
                ft_scaler.scale(loss_ft).backward()
                ft_scaler.unscale_(ft_optimizer)
                torch.nn.utils.clip_grad_norm_([p for g in ft_optimizer.param_groups for p in g["params"] if p.grad is not None], cfg["grad_clip"])
                ft_scaler.step(ft_optimizer)
                ft_scaler.update()

    # ---- update replay memory with new task examples (on CPU) ----
    # prefer using replay.add_samples if it exists
    try:
        replay.add_samples(t_id, train_loader, max_samples=cfg["replay_per_task"])
    except Exception:
        # fallback: collect from loader manually
        imgs_accum, labs_accum = [], []
        for imgs_b, labs_b in train_loader:
            imgs_accum.append(imgs_b.cpu())
            labs_accum.append(labs_b.cpu())
        if len(imgs_accum) > 0:
            imgs_all = torch.cat(imgs_accum, dim=0)
            labs_all = torch.cat(labs_accum, dim=0)
            idxs = torch.randperm(len(labs_all))[:cfg["replay_per_task"]]
            replay.data = [imgs_all[i] for i in idxs.tolist()]
            replay.targets = [int(l) for l in labs_all[idxs].tolist()]
            replay.task_ids = [t_id] * len(idxs)

    # ---- snapshot model to CPU for next task teacher (deep copy) ----
    torch.cuda.empty_cache(); gc.collect()
    with torch.no_grad():
            old_model_next = deepcopy_model_to_cpu(model)
    accs = []
    model.to(device)
    for vid in range(t_id + 1):
        model.set_task(vid)
        acc = evaluate_task(model, val_loaders[vid], device)
        accs.append(acc)
    return old_model_next, accs


In [19]:
replay = ReplayBuffer(capacity_per_task=CONFIG["replay_per_task"])

# datasets per task (example)
# train_loaders = [train_loader_task0, train_loader_task1, train_loader_task2, ...]
# val_loaders =   [val_loader_task0,   val_loader_task1,   val_loader_task2,   ...]

# for the first iteration, there’s no teacher model yet
old_model_cpu = None

# to record accuracy history
taskwise_accs = []
# Loop through each incremental task
for t_id in range(len(task_loaders)):
    print(f"\n===== Training on Task {t_id} =====")

    # call the training method
    old_model_cpu, accs = train_task_with_replay_and_distill(
        model=model,
        t_id=t_id,
        train_loader=task_loaders[t_id],
        val_loaders=val_loaders,
        replay=replay,
        old_model_cpu=old_model_cpu,
        compute_metrics=None,   # optional
        cfg=CONFIG,
        device=device
    )

    print(f"Task {t_id} accuracies (all seen tasks): {accs}")
    taskwise_accs.append(accs)


/tmp/ipykernel_160/1384571469.py:182: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()



===== Training on Task 0 =====


Task 0 | Epoch 1/10:   0%|          | 0/188 [00:00<?, ?it/s]/tmp/ipykernel_160/1384571469.py:202: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Task 0 | Epoch 1/10: 100%|██████████| 188/188 [02:01<00:00,  1.54it/s, Acc=68.51%, Loss=0.6089]


Task 0 | Epoch 1/10 | Loss=0.6089 | Acc=68.51%


Task 0 | Epoch 2/10: 100%|██████████| 188/188 [01:41<00:00,  1.85it/s, Acc=76.40%, Loss=0.5375]


...

Task 5 | Epoch 8/10 | Loss=0.1985 | Acc=100.00%


Task 5 | Epoch 9/10: 100%|██████████| 240/240 [04:20<00:00,  1.08s/it, Acc=100.00%, Loss=0.1985]


Task 5 | Epoch 9/10 | Loss=0.1985 | Acc=100.00%


Task 5 | Epoch 10/10: 100%|██████████| 240/240 [04:20<00:00,  1.09s/it, Acc=100.00%, Loss=0.1985]


Task 5 | Epoch 10/10 | Loss=0.1985 | Acc=100.00%
Task 5 accuracies (all seen tasks): [76.45, 76.5, 89.8854961832061, 99.92163009404389, 92.51386321626617, 99.92163009404389]


In [20]:
print("Final per-task accuracies:")
for i, accs in enumerate(taskwise_accs):
    print(f"After Task {i}: {accs}")


Final per-task accuracies:
After Task 0: [93.5]
After Task 1: [84.7, 90.75]
After Task 2: [88.4, 91.875, 91.22137404580153]
After Task 3: [73.75, 73.0, 86.45038167938931, 99.96081504702194]
After Task 4: [80.7, 86.5, 91.22137404580153, 100.0, 95.10166358595194]
After Task 5: [76.45, 76.5, 89.8854961832061, 99.92163009404389, 92.51386321626617, 99.92163009404389]


In [22]:
last_list = taskwise_accs[-1]  # get the most recent task's accuracy list
avg_last = sum(last_list) / len(last_list)
print(f"\nAverage Accuracy for last task: {avg_last:.2f}%")



Average Accuracy for last task: 89.20%


In [29]:
import pickle
save_dir = "/kaggle/working/checkpoints"
os.makedirs(save_dir, exist_ok=True)

model_path = os.path.join(save_dir, "final_model.pth")
replay_path = os.path.join(save_dir, "replay_buffer.pkl")

torch.save(model.state_dict(), model_path)



print("✅ Model and replay buffer saved successfully!")
print("➡ You can now download them from the 'Data' > 'Output' section in Kaggle.")



✅ Model and replay buffer saved successfully!
➡ You can now download them from the 'Data' > 'Output' section in Kaggle.


In [30]:
from IPython.display import FileLink

# Make downloadable links
FileLink("./checkpoints/final_model.pth")





/kaggle/working/checkpoints/final_model.pth